In [139]:
import copy, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

file_name = 'davidson'
csv_path = './team_adv_stats_avg/' + str(file_name) + '.csv'

In [140]:
df = pd.read_csv('./team_stats/' + str(file_name) + '_adv_2023.csv')

df.drop(['W/L'], axis=1, inplace=True)

df = df.loc[:, ~df.columns.str.startswith('Unnamed')]

In [141]:
df

,Date,Opp,Tm,Opp.1,ORtg,DRtg,Pace,FTr,3PAr,TS%,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1
0,2022-11-07,Guilford,87,64,114.5,84.2,76.1,0.725,0.667,0.634,...,11.8,2.2,0.598,14.9,16.7,0.510,0.394,15.3,68.1,0.113
1,2022-11-09,Wright State,102,97,109.7,104.3,74.1,0.537,0.343,0.606,...,4.3,3.4,0.597,16.0,23.7,0.328,0.489,11.2,69.8,0.125
2,2022-11-13,VMI,75,71,108.7,102.9,69.4,0.288,0.525,0.559,...,7.2,2.8,0.525,10.7,17.6,0.220,0.538,15.1,83.3,0.264
3,2022-11-17,College of Charleston,66,89,93.0,125.4,70.5,0.510,0.216,0.521,...,9.9,5.0,0.471,20.2,25.8,0.353,0.542,12.4,48.6,0.153
4,2022-11-18,South Carolina,69,60,98.6,85.7,69.7,0.155,0.431,0.554,...,10.0,2.6,0.526,17.3,14.8,0.138,0.446,19.2,70.6,0.179
5,2022-11-20,Old Dominion,66,61,98.5,91.0,66.6,0.489,0.319,0.570,...,3.0,5.5,0.511,19.5,24.0,0.383,0.420,10.0,64.9,0.043
6,2022-11-25,San Francisco,89,80,129.0,115.9,69.1,0.717,0.283,0.722,...,8.7,3.4,0.663,14.0,10.5,0.609,0.558,14.9,60.0,0.217
7,2022-11-29,Charlotte,66,68,100.0,103.0,58.6,0.327,0.462,0.549,...,9.1,6.1,0.519,13.0,7.1,0.231,0.543,12.5,78.1,0.086
8,2022-12-03,Delaware,67,69,103.1,106.2,64.7,0.125,0.328,0.494,...,7.7,2.6,0.469,6.9,17.6,0.109,0.542,9.6,68.8,0.085
9,2022-12-07,Western Carolina,72,64,98.6,87.7,73.8,0.418,0.291,0.546,...,11.0,0.0,0.509,17.5,26.7,0.291,0.482,20.0,82.1,0.200


In [142]:
averages_df = df.copy()

for i in range(4, len(df)):
    averages_df.loc[i, df.columns[2:]] = df.loc[i-4:i, df.columns[1:]].mean()

averages_df

C:\Users\lakea\AppData\Local\Temp\ipykernel_17688\2049136851.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  averages_df.loc[i, df.columns[2:]] = df.loc[i-4:i, df.columns[1:]].mean()


,Date,Opp,Tm,Opp.1,ORtg,DRtg,Pace,FTr,3PAr,TS%,...,STL%,BLK%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1
0,2022-11-07,Guilford,87.0,64.0,114.50,84.20,76.10,0.7250,0.6670,0.6340,...,11.80,2.20,0.5980,14.90,16.70,0.5100,0.3940,15.30,68.10,0.1130
1,2022-11-09,Wright State,102.0,97.0,109.70,104.30,74.10,0.5370,0.3430,0.6060,...,4.30,3.40,0.5970,16.00,23.70,0.3280,0.4890,11.20,69.80,0.1250
2,2022-11-13,VMI,75.0,71.0,108.70,102.90,69.40,0.2880,0.5250,0.5590,...,7.20,2.80,0.5250,10.70,17.60,0.2200,0.5380,15.10,83.30,0.2640
3,2022-11-17,College of Charleston,66.0,89.0,93.00,125.40,70.50,0.5100,0.2160,0.5210,...,9.90,5.00,0.4710,20.20,25.80,0.3530,0.5420,12.40,48.60,0.1530
4,2022-11-18,South Carolina,79.8,76.2,104.90,100.50,71.96,0.4430,0.4364,0.5748,...,8.64,3.20,0.5434,15.82,19.72,0.3098,0.4818,14.64,68.08,0.1668
5,2022-11-20,Old Dominion,75.6,75.6,101.70,101.86,70.06,0.3958,0.3668,0.5620,...,6.88,3.86,0.5260,16.74,21.18,0.2844,0.4870,13.58,67.44,0.1528
6,2022-11-25,San Francisco,73.0,72.2,105.56,104.18,69.06,0.4318,0.3548,0.5852,...,7.76,3.86,0.5392,16.34,18.54,0.3406,0.5008,14.32,65.48,0.1712
7,2022-11-29,Charlotte,71.2,71.6,103.82,104.20,66.90,0.4396,0.3422,0.5832,...,8.14,4.52,0.5380,16.80,16.44,0.3428,0.5018,13.80,64.44,0.1356
8,2022-12-03,Delaware,71.4,67.6,105.84,100.36,65.74,0.3626,0.3646,0.5778,...,7.70,4.04,0.5376,14.14,14.80,0.2940,0.5018,13.24,68.48,0.1220
9,2022-12-07,Western Carolina,72.0,68.4,105.84,100.76,66.56,0.4152,0.3366,0.5762,...,7.90,3.52,0.5342,14.18,17.18,0.3246,0.5090,13.40,70.78,0.1262


In [143]:
averages_df.to_csv(csv_path)